# Objective

Use ACO to optimize Logistic Regression hyperparameters (`C`) for the breast cancer classification dataset.

**Why ACO?**
- ACO is inspired by ant foraging behavior, where ants explore and deposit pheromones to find optimal paths. It’s well-suited for global optimization problems and can efficiently navigate large search spaces.

`Import Libraries`


In [7]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


`Load and Preprocess Data`

In [11]:
df = pd.read_csv('C:/Users/ajayr/Desktop/Projects to upload/Metaheuristic_Optimization_for_Logistic_Regression/data/data.csv')

# Drop irrelevant columns
if 'id' in df.columns:
    df.drop(columns=['id'], inplace=True)

# Encode target
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

X = df.drop(columns=['diagnosis'])
y = df['diagnosis']

# Scale features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y)

`Define Fitness Function`

In [16]:
def fitness(C_value):
    model = LogisticRegression(C=C_value, max_iter=1000, solver='liblinear')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

`Implement ACO`

In [19]:
# ACO Parameters
num_ants = 10
num_iterations = 20
lb, ub = 0.01, 100
evaporation_rate = 0.5
pheromone = np.ones(10)  # initial pheromone levels for discretized choices
discretized_C = np.linspace(lb, ub, 10)  # discretized search space

In [21]:
best_score = 0
best_C = None

start_time = time.time()

for iteration in range(num_iterations):
    ant_solutions = []
    ant_scores = []
    
    for _ in range(num_ants):
        # Probability distribution based on pheromone
        probs = pheromone / pheromone.sum()
        idx = np.random.choice(range(len(discretized_C)), p=probs)
        C_value = discretized_C[idx]
        
        score = fitness(C_value)
        ant_solutions.append(idx)
        ant_scores.append(score)
    
    # Update pheromone based on scores
    pheromone = (1 - evaporation_rate) * pheromone
    for idx, score in zip(ant_solutions, ant_scores):
        pheromone[idx] += score
    
    # Track best solution
    max_idx = np.argmax(ant_scores)
    if ant_scores[max_idx] > best_score:
        best_score = ant_scores[max_idx]
        best_C = discretized_C[ant_solutions[max_idx]]

end_time = time.time()
execution_time = end_time - start_time

In [23]:
print(f"Best C found by ACO: {best_C:.4f}")
print(f"Best Accuracy by ACO: {best_score:.4f}")
print(f"ACO Execution Time: {execution_time:.4f} seconds")

Best C found by ACO: 11.1200
Best Accuracy by ACO: 0.9825
ACO Execution Time: 0.3714 seconds


# Conclusion

The Ant Colony Optimization approach delivered 98.25% accuracy, matching PSO’s performance and outperforming both the baseline Logistic Regression (97%) and GridSearchCV (97.37%). With an execution time of 0.3714 seconds, ACO proved both accurate and efficient, making it an excellent alternative for hyperparameter optimization when computational speed is a priority. Compared to GridSearchCV, ACO avoids exhaustive search and finds near-optimal solutions quickly, showcasing the power of metaheuristic algorithms in real-world machine learning workflows.

# Summary

- Implemented **Ant Colony Optimization (ACO)** to optimize Logistic Regression hyperparameter `C`.
- **Best C:** 11.1200
- **Accuracy:** 98.25% (same as PSO and higher than GridSearchCV).
- **Execution Time:** 0.3714 seconds (much faster than GridSearchCV and close to PSO).
- **Insight:** ACO demonstrated excellent accuracy with efficient execution, reinforcing the effectiveness of metaheuristic optimization for hyperparameter tuning.